In [1]:
# pip install torchsummary


In [2]:
from torchsummary import summary

In [3]:
import os
import csv
import numpy as np
import scipy.io as sio

from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import RFE
from nilearn import connectome

from scipy.spatial import distance


# Reading and computing the input data

# Selected pipeline
pipeline = 'cpac'

# Input data variables
root_folder = '../ABIDE/'
data_folder = os.path.join(root_folder, 'ABIDE_pcp/cpac/filt_noglobal')
phenotype = os.path.join(root_folder, 'ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv')


# Get timeseries arrays for list of subjects
def get_timeseries(subject_list, atlas_name):
    """
        subject_list : list of short subject IDs in string format
        atlas_name   : the atlas based on which the timeseries are generated e.g. aal, cc200

    returns:
        time_series  : list of timeseries arrays, each of shape (timepoints x regions)
    """

    timeseries = []
    for i in range(len(subject_list)):
        subject_folder = os.path.join(data_folder, subject_list[i])
        ro_file = [f for f in os.listdir(subject_folder) if f.endswith('_rois_' + atlas_name + '.1D')]
        fl = os.path.join(subject_folder, ro_file[0])
#         print("Reading timeseries file %s" %fl)
        timeseries.append(np.loadtxt(fl, skiprows=0))

    return timeseries


# Compute connectivity matrices
def subject_connectivity(timeseries, subject, atlas_name, kind, save=True, save_path=data_folder):
    """
        timeseries   : timeseries table for subject (timepoints x regions)
        subject      : the subject ID
        atlas_name   : name of the parcellation atlas used
        kind         : the kind of connectivity to be used, e.g. lasso, partial correlation, correlation
        save         : save the connectivity matrix to a file
        save_path    : specify path to save the matrix if different from subject folder

    returns:
        connectivity : connectivity matrix (regions x regions)
    """

    print("Estimating %s matrix for subject %s" % (kind, subject))

    if kind in ['tangent', 'partial correlation', 'correlation']:
        conn_measure = connectome.ConnectivityMeasure(kind=kind)
        connectivity = conn_measure.fit_transform([timeseries])[0]

    if save:
        subject_file = os.path.join(save_path, subject,
                                    subject + '_' + atlas_name + '_' + kind.replace(' ', '_') + '.mat')
        sio.savemat(subject_file, {'connectivity': connectivity})

    return connectivity

subID = []
# Get the list of subject IDs
def get_ids(num_subjects=None):
    
    subject_IDs = np.genfromtxt(os.path.join(data_folder, 'subject_IDs.txt'), dtype=str)
    print(subject_IDs)
    if num_subjects is not None:
        subject_IDs = subject_IDs[:num_subjects]
    global subID    
    subID=subject_IDs; 
    print(subID);
    return subject_IDs


# Get phenotype values for a list of subjects
def get_subject_score(subject_list, score):
    scores_dict = {}

    with open(phenotype) as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            if row['SUB_ID'] in subject_list:
                scores_dict[row['SUB_ID']] = row[score]

    return scores_dict


# Dimensionality reduction step for the feature vector using a ridge classifier
def feature_selection(matrix, labels, train_ind, fnum):
    """
        matrix       : feature matrix (num_subjects x num_features)
        labels       : ground truth labels (num_subjects x 1)
        train_ind    : indices of the training samples
        fnum         : size of the feature vector after feature selection

    return:
        x_data      : feature matrix of lower dimension (num_subjects x fnum)
    """

    estimator = RidgeClassifier()
    selector = RFE(estimator, n_features_to_select=fnum, step=100, verbose=1)

    featureX = matrix[train_ind, :]
    featureY = labels[train_ind]
  
    selector = selector.fit(featureX, featureY.ravel())
   
    x_data = selector.transform(matrix)

    print("Number of labeled samples %d" % len(train_ind))
    print("Number of features selected %d" % x_data.shape[1])
#     print("x_data :",x_data.shape,len(x_data))
    return x_data


# Load precomputed fMRI connectivity networks
def get_networks(subject_list, kind, atlas_name="aal", variable='connectivity'):
    """
        subject_list : list of subject IDs
        kind         : the kind of connectivity to be used, e.g. lasso, partial correlation, correlation
        atlas_name   : name of the parcellation atlas used
        variable     : variable name in the .mat file that has been used to save the precomputed networks


    return:
        matrix      : feature matrix of connectivity networks (num_subjects x network_size)
    """

    all_networks = []
#     subject_list = subject_list[:1]
    for subject1 in subject_list:
        fl = f'../Datasets/all_fc_matrix_rois_ez/matrix_rois_ez_{subject1}.mat'
        try:  
            matrix = sio.loadmat(fl)[variable]
            all_networks.append(matrix)
        except FileNotFoundError:
            fl = f'../Datasets/all_fc_matrix_rois_ez/matrix_rois_ez_{50002}.mat'
            matrix = sio.loadmat(fl)[variable]
            all_networks.append(matrix)
            
            
    # all_networks=np.array(all_networks)

    idx = np.triu_indices_from(all_networks[0], 1)
#     print("idx :- ",idx)
    norm_networks = [np.arctanh(mat) if not np.all(np.abs(mat) == 1) else mat for mat in all_networks]
#     print("norm_networks : ",len(norm_networks),len(norm_networks[0]),len(norm_networks[0][0]))
    vec_networks = [mat[idx] for mat in norm_networks]
#     print(len(vec_networks),len(vec_networks[0]))
#     print("vec_networks : ",vec_networks)
    matrix = np.vstack(vec_networks)
    return matrix


# Construct the adjacency matrix of the population from phenotypic scores
def create_affinity_graph_from_scores(scores, pd_dict):
    num_nodes = len(pd_dict[scores[0]]) 
    graph = np.zeros((num_nodes, num_nodes))

    for l in scores:
        label_dict = pd_dict[l]

        if l in ['AGE_AT_SCAN', 'FIQ']:
            for k in range(num_nodes):
                for j in range(k + 1, num_nodes):
                    try:
                        val = abs(float(label_dict[k]) - float(label_dict[j]))
                        if val < 2:
                            graph[k, j] += 1
                            graph[j, k] += 1
                    except ValueError:  # missing label
                        pass

        else:
            for k in range(num_nodes):
                for j in range(k + 1, num_nodes):
                    if label_dict[k] == label_dict[j]:
                        graph[k, j] += 1
                        graph[j, k] += 1

    return graph

def get_static_affinity_adj(features, pd_dict):
    pd_affinity = create_affinity_graph_from_scores(['SEX', 'SITE_ID'], pd_dict) 
    distv = distance.pdist(features, metric='correlation')
#     print("distv : ",len(distv),distv.shape);
    dist = distance.squareform(distv)  
    sigma = np.mean(dist)
    feature_sim = np.exp(- dist ** 2 / (2 * sigma ** 2))
#     print("feature_sim : ",feature_sim.shape)
    adj = pd_affinity * feature_sim  
#     print("adj : ",adj.shape)
#     print(adj);
    return adj


In [4]:
import numpy as np
import scipy.sparse as sp
import torch


from sklearn.model_selection import StratifiedKFold
from scipy.spatial import distance
from scipy.sparse.linalg.eigen import eigsh

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def preprocess_features(features):
    """Row-normalize feature matrix"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features

C:\Users\hpw\AppData\Local\Temp\ipykernel_32700\3783650984.py:8: DeprecationWarning: Please import `eigsh` from the `scipy.sparse.linalg` namespace; the `scipy.sparse.linalg.eigen` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.sparse.linalg.eigen import eigsh


In [5]:
import numpy as np
import torch
# from utils import preprocess_features
from sklearn.model_selection import StratifiedKFold


class dataloader():
    def __init__(self): 
        self.pd_dict = {}
        self.node_ftr_dim = 2000
        self.num_classes = 2 

    def load_data(self, params, connectivity='correlation', atlas='ho'):
        ''' load multimodal data from ABIDE
        return: imaging features (raw), labels, non-image data
        '''
        subject_IDs = get_ids()
#         print(len(subject_IDs ))
        labels = get_subject_score(subject_IDs, score='DX_GROUP')
        num_nodes =len(subject_IDs)

        sites = get_subject_score(subject_IDs, score='SITE_ID')
        unique = np.unique(list(sites.values())).tolist()
        ages = get_subject_score(subject_IDs, score='AGE_AT_SCAN')
        genders = get_subject_score(subject_IDs, score='SEX') 

        y_onehot = np.zeros([num_nodes, self.num_classes])
        y = np.zeros([num_nodes])
        site = np.zeros([num_nodes], dtype=int)
        age = np.zeros([num_nodes], dtype=np.float32)
        gender = np.zeros([num_nodes], dtype=int)
        for i in range(num_nodes):
            y_onehot[i, int(labels[subject_IDs[i]])-1] = 1
            y[i] = int(labels[subject_IDs[i]])
            site[i] = unique.index(sites[subject_IDs[i]])
            age[i] = float(ages[subject_IDs[i]])
            gender[i] = genders[subject_IDs[i]]
        
        self.y = y -1  

        self.raw_features = get_networks(subject_IDs, kind=connectivity, atlas_name=atlas)

        phonetic_data = np.zeros([num_nodes, 3], dtype=np.float32)
        phonetic_data[:,0] = site 
        phonetic_data[:,1] = gender 
        phonetic_data[:,2] = age 

        self.pd_dict['SITE_ID'] = np.copy(phonetic_data[:,0])
        self.pd_dict['SEX'] = np.copy(phonetic_data[:,1])
        self.pd_dict['AGE_AT_SCAN'] = np.copy(phonetic_data[:,2]) 
        
        return self.raw_features, self.y, phonetic_data

    def data_split(self, n_folds):
#         split data by k-fold CV
        skf = StratifiedKFold(n_splits=n_folds)
#         print("skf : ",skf)
        cv_splits = list(skf.split(self.raw_features, self.y))
#         cv_splits_array = np.array(skf.split(self.raw_features, self.y))
#         print(cv_splits_array.shape,len(cv_splits_array )) 
#         print(raw_features)
#         print(cv_splits)
#         print("cv_splits : ",len(cv_splits))
#         print(": ",len(cv_splits[0]))
#         print(": ",len(cv_splits[0][0]))
#         x=len(cv_splits);
#         for i in range(x):
#             y=len(cv_splits[i]);
#             for j in range(y):
#                 z=len(cv_splits[i][j])
#                 print(z)
            
#         print("cv_splits : ",cv_splits  )
        return cv_splits 

    def get_node_features(self, train_ind):
        '''preprocess node features for wl-deepgcn
        '''
        node_ftr = feature_selection(self.raw_features, self.y, train_ind, self.node_ftr_dim)
        self.node_ftr = preprocess_features(node_ftr) 
        return self.node_ftr

    def get_WL_inputs(self, nonimg):
        '''get WL inputs for wl-deepgcn 
        '''
        # construct edge network inputs 
        n = self.node_ftr.shape[0] 
        
        num_edge = n*(1+n)//2 - n  # n*(n-1)//2,HO=6105
#         print("n :",n ,num_edge)
        pd_ftr_dim = nonimg.shape[1]
#         print("pd_ftr_dim :",pd_ftr_dim)
        edge_index = np.zeros([2, num_edge], dtype=np.int64) 
#         print("edge_index : ",edge_index)
        edgenet_input = np.zeros([num_edge, 2*pd_ftr_dim], dtype=np.float32)  
#         print("edgenet_input : ",edgenet_input)
        aff_score = np.zeros(num_edge, dtype=np.float32)
#         print("aff_score :",aff_score,len(aff_score))
        # static affinity score used to pre-prune edges 
#         print("pd_dict : ",len(self.pd_dict))
        aff_adj = get_static_affinity_adj(self.node_ftr, self.pd_dict)  
        flatten_ind = 0 
        for i in range(n):
            for j in range(i+1, n):
                edge_index[:,flatten_ind] = [i,j]
                edgenet_input[flatten_ind]  = np.concatenate((nonimg[i], nonimg[j]))
                aff_score[flatten_ind] = aff_adj[i][j]  
                flatten_ind +=1

        assert flatten_ind == num_edge, "Error in computing edge input"
        
        keep_ind = np.where(aff_score > 1.1)[0]  
        edge_index = edge_index[:, keep_ind]
        edgenet_input = edgenet_input[keep_ind]
        
#         print(edge_index.shape ,edgenet_input.shape);
#         print("edge_index : ",edge_index)
#         print("edgenet_input : ",edgenet_input)

        return edge_index, edgenet_input
    

In [6]:
import torch
from torch.nn import Linear as Lin, Sequential as Seq
import torch.nn.functional as F
from torch import nn

class WL(torch.nn.Module):
    def __init__(self, input_dim, dropout=0.3):
        print("welcome __init__")
        super(WL, self).__init__()
        print(input_dim)
        h1=256
        h2=128
        self.parser =nn.Sequential(
                nn.Linear(input_dim, h1, bias=True),
                nn.LeakyReLU(inplace=True),
                nn.BatchNorm1d(h1),
                nn.Dropout(dropout),
                nn.Linear(h1, h2, bias=True),
                nn.LeakyReLU(inplace=True),
                nn.BatchNorm1d(h2),
                nn.Dropout(dropout),
                nn.Linear(h2, h2, bias=True),
                )
        print("after sequence :");
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-8)
        self.input_dim = input_dim
        self.model_init()
        print("hello ,after init")
        self.relu = nn.ReLU(inplace=True)
        self.elu = nn.ReLU()

    def forward(self, x):
        x1 = x[:,0:self.input_dim]
        x2 = x[:,self.input_dim:]
        print("x: ",len(x))
        print(x)
    
        h1 = self.parser(x1) 
        h2 = self.parser(x2) 
#         print("h1 : ",len(h1),h1);
#         print("h2 : ",len(h2),h2);
        p = (self.cos(h1,h2) + 1)*0.5
#         print("p : ",len(p))
        return p

    def model_init(self):
        for m in self.modules():
            if isinstance(m, Lin):
                torch.nn.init.kaiming_normal_(m.weight)
                m.weight.requires_grad = True
                if m.bias is not None:
                    m.bias.data.zero_()
                    m.bias.requires_grad = True



In [7]:
# !pip install torchvision
# from torchvision.models import models

In [8]:
from torchvision import models

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import Linear as Lin, Sequential as Seq
import torch_geometric as tg

class MLP(nn.Module):
    def __init__(self, input_dim, nhid):
        super(MLP,self).__init__()
        self.cls = nn.Sequential(
            torch.nn.Linear(input_dim,nhid))
        
    def forward(self, features):
        output = self.cls(features)
        return output
nhid1=1
input_dim1=1
global nhid1
global input_dim1
class GCN(nn.Module):
    def __init__(self, input_dim, nhid, num_classes, ngl, dropout, edge_dropout, edgenet_input_dim):
        super(GCN, self).__init__()
        K=3   
        hidden = [nhid for i in range(ngl)] 
        self.dropout = dropout 
        self.edge_dropout = edge_dropout 
        bias = False 
        self.relu = torch.nn.ReLU(inplace=True) 
        self.ngl = ngl 
        self.gconv = nn.ModuleList()
        nhid1=nhid
        input_dim1=input_dim
        print("input_dim : ",input_dim)
        for i in range(ngl):
            in_channels = input_dim if i==0  else hidden[i-1]
            self.gconv.append(tg.nn.ChebConv(in_channels, hidden[i], K, normalization='sym', bias=bias)) 
          
        self.cls = nn.Sequential(
                torch.nn.Linear(16, 128),
                torch.nn.ReLU(inplace=True),
                nn.BatchNorm1d(128), 

            torch.nn.Linear(128, num_classes))
        print("input_dim : ",input_dim,edgenet_input_dim) 
        self.edge_net = WL(input_dim=edgenet_input_dim//2, dropout=dropout)
#         print(model12) 
        self.model_init()

    def model_init(self):
        for m in self.modules():
            if isinstance(m, Lin):
                torch.nn.init.kaiming_normal_(m.weight) # He init
                m.weight.requires_grad = True
                if m.bias is not None:
                    m.bias.data.zero_()
                    m.bias.requires_grad = True

    def forward(self, features, edge_index, edgenet_input, enforce_edropout=False): 
#         if self.edge_dropout>0:
#             if enforce_edropout or self.training:
#                 one_mask = torch.ones([edgenet_input.shape[0],1])
#                 self.drop_mask = F.dropout(one_mask, self.edge_dropout, True)
#                 self.bool_mask = torch.squeeze(self.drop_mask.type(torch.bool))
#                 edge_index = edge_index[:, self.bool_mask] 
#                 edgenet_input = edgenet_input[self.bool_mask] # Weights
            
        edge_weight = torch.squeeze(self.edge_net(edgenet_input))
        

        # GCN residual connection
        # input layer
        print("0 :" ,features.shape)
        features = F.dropout(features, self.dropout, self.training)
        print("1 :" ,features.shape)
        x = self.relu(self.gconv[0](features, edge_index, edge_weight)) 
        
        x_temp = x
        print("2 :" ,x_temp.shape,features.shape)
        # hidden layers
        for i in range(1, self.ngl - 1): # self.ngl→7
            x = F.dropout(x_temp, self.dropout, self.training)
            print(i," : ",x.shape)
            x = self.relu(self.gconv[i](x, edge_index, edge_weight)) 
            x_temp = x_temp + x # ([871,64])
            print(i," -: ",x_temp.shape)

        # output layer
        x = F.dropout(x_temp, self.dropout, self.training)
        x = self.relu(self.gconv[self.ngl - 1](x, edge_index, edge_weight))
        x_temp = x_temp + x
        
        output = x # Final output is not cumulative
        print(x.shape);
        output = self.cls(output) 
    
        print("output : ",output);
        print("edge_weight : ",edge_weight);
        return output, edge_weight
    

    



In [10]:
import pandas as pd
import numpy as np

In [11]:
import torch
import torchmetrics
from torchmetrics.classification import MulticlassSpecificity
import matplotlib.pyplot as plt
import numpy as np
import itertools

from sklearn.metrics import precision_recall_fscore_support

def torchmetrics_accuracy(preds, labels):
#     print("preds : ",preds)
    acc = torchmetrics.functional.accuracy(preds, labels,task="multiclass", num_classes=1)
    return acc

def torchmetrics_spef(preds, labels):
    metric = MulticlassSpecificity(num_classes=2)
    spef = metric(preds, labels)
    return spef

def torchmetrics_auc(preds, labels):
    auc = torchmetrics.functional.auroc(preds, labels, task="multiclass", num_classes=2)
    return auc

def confusion_matrix(preds, labels):
    conf_matrix = torch.zeros(2, 2)
    preds = torch.argmax(preds, 1)
    for p, t in zip(preds, labels):
        conf_matrix[t, p] += 1 
    return conf_matrix
def plot_confusion_matrix(cm, normalize=False, title='Confusion matrix', cmap=plt.cm.Oranges):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Input
    - cm : computer the value of confusion matrix
    - normalize : True: %, False: 123
    """
    classes = ['0:ASD','1:TC']
    if normalize:
        cm = cm.numpy()
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else '.0f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def correct_num(preds, labels):
    """Accuracy, auc with masking.Acc of the masked samples"""
    correct_prediction = np.equal(np.argmax(preds, 1), labels).astype(np.float32)
    return np.sum(correct_prediction)

def prf(preds, labels, is_logit=True):
    ''' input: logits, labels  ''' 
    pred_lab= np.argmax(preds, 1)
    p,r,f,s  = precision_recall_fscore_support(labels, pred_lab, average='binary')
    return [p,r,f]

In [12]:
import argparse
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()

In [13]:
output = []
valid_ind = []
global output
global valid_ind 

In [14]:
from __future__ import division
from __future__ import print_function

import os
import time
import argparse
import numpy as np
import io
import sys

import torch
import torch.optim as optim



# from dataloader import dataloader

from sklearn.model_selection import train_test_split
from sklearn.metrics import auc

if hasattr(sys.stdout, 'buffer'):
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')


class Args:
    def __init__(self):
        self.no_cuda = False
        self.seed = 46
        self.epochs = 2
        self.lr = 0.001
        self.weight_decay = 5e-5
        self.hidden = 16
        self.dropout = 0.2
        self.atlas = 'ez'
        self.num_features = 2000
        self.folds = 2
        self.connectivity = 'correlation'
        self.max_degree = 3
        self.ngl = 8
        self.edropout = 0.3
        self.train = 0
        self.ckpt_path = '../folds/rois_ez_pth_b'
        self.early_stopping = True
        self.early_stopping_patience = 20

# Instantiate Args class
args = Args()

# Check if CUDA is available
args.cuda = not args.no_cuda and torch.cuda.is_available()

# Set random seeds
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Create params dictionary
params = vars(args)

# print Hyperparameters
print('Hyperparameters:')
for key, value in params.items():
    print(key + ":", value)

corrects = np.zeros(args.folds, dtype=np.int32) 
accs = np.zeros(args.folds, dtype=np.float32) 
aucs = np.zeros(args.folds, dtype=np.float32)
prfs = np.zeros([args.folds,3], dtype=np.float32) # Save Precision, Recall, F1
test_num = np.zeros(args.folds, dtype=np.float32)


print('  Loading dataset ...')
dataloader = dataloader()
raw_features, y, nonimg = dataloader.load_data(params) 
cv_splits = dataloader.data_split(args.folds)
print("y : ",y)
print(subID)
features=raw_features

t1 = time.time()
count=1;
for i in range(1):
    t_start = time.time()
    train_ind, test_ind = cv_splits[i]
    print("train_ind , test_ind : ",len(train_ind),len(test_ind))
    train_ind, valid_ind = train_test_split(train_ind, test_size=0.1, random_state = 24)
#     valid_ind =  valid_ind  + test_ind; 
    print(valid_ind)
#     for i in valid_ind:
#         print(subID[i-1]," ")
    cv_splits[i] = (train_ind, valid_ind)
    cv_splits[i] = cv_splits[i] + (test_ind,)
    print(test_ind)
   
    print('Size of the {}-fold Training, Validation, and Test Sets:{},{},{}' .format(i+1, len(cv_splits[i][0]), len(cv_splits[i][1]), len(cv_splits[i][2])))

    if args.train == 1:
        
        for j in range(1):
            print(' Starting the {}-{} Fold:：'.format(i+1,j+1))
            node_ftr = dataloader.get_node_features(train_ind)
        
            print("==",node_ftr.shape);
            edge_index, edgenet_input = dataloader.get_WL_inputs(nonimg)
            edgenet_input = (edgenet_input - edgenet_input.mean(axis=0)) / edgenet_input.std(axis=0)
            
            model = GCN(input_dim = args.num_features,
                        nhid = args.hidden, 
                        num_classes = 2, 
                        ngl = args.ngl, 
                        dropout = args.dropout, 
                        edge_dropout = args.edropout, 
                        edgenet_input_dim = 2*nonimg.shape[1])
            optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
            input_dim=args.num_features
            
#             if args.cuda:
            model
            features = torch.tensor(node_ftr, dtype=torch.float32)
            edge_index = torch.tensor(edge_index, dtype=torch.long)
            edgenet_input = torch.tensor(edgenet_input, dtype=torch.float32)
            labels = torch.tensor(y, dtype=torch.long)
            fold_model_path = args.ckpt_path + "/fold{}.pth".format(i+1)
           
    
            acc = 0
            best_val_loss = float('inf') # early stoppping: Initialized to positive infinity
            current_patience = 0 # early stopping: Used to record the epochs of the current early stopping
            
            epoch_store = []
            acc_train_store =[]        
            pre_train_store =[]
            recall_train_store =[]
            F1_train_store =[]
            AUC_train_store =[]
            acc_val_store=[]
            pre_val_store=[]
            recall_val_store=[]
            F1_val_store=[]
            AUC_val_store=[]
            
            for epoch in range(args.epochs):
                # train
                model.train()
                with torch.set_grad_enabled(True):
                    optimizer.zero_grad()
                    output, edge_weights = model(features, edge_index, edgenet_input)
#                     print("output[train_ind] : ",output[train_ind],labels[train_ind])
                    loss_train = torch.nn.CrossEntropyLoss()(output[train_ind], labels[train_ind])
                    loss_train.backward()
                    optimizer.step()
                acc_train = torchmetrics_accuracy(output[train_ind], labels[train_ind])
                auc_train = torchmetrics_auc(output[train_ind], labels[train_ind])
                logits_train = output[train_ind].detach().cpu().numpy()
                prf_train = prf(logits_train, y[train_ind])

                
                # valid
                model.eval()
                with torch.set_grad_enabled(False):
                    output, edge_weights = model(features, edge_index, edgenet_input)
                loss_val = torch.nn.CrossEntropyLoss()(output[valid_ind], labels[valid_ind])
                acc_val = torchmetrics_accuracy(output[valid_ind], labels[valid_ind])
                auc_val = torchmetrics_auc(output[valid_ind], labels[valid_ind])
                logits_val = output[valid_ind].detach().cpu().numpy()
                prf_val = prf(logits_val, y[valid_ind])

                
                print('Epoch:{:04d}'.format(epoch+1))
                print('acc_train:{:.4f}'.format(acc_train),
                      'pre_train:{:.4f}'.format(prf_train[0]),
                      'recall_train:{:.4f}'.format(prf_train[1]),
                      'F1_train:{:.4f}'.format(prf_train[2]),
                      'AUC_train:{:.4f}'.format(auc_train))
                print('acc_val:{:.4f}'.format(acc_val),
                      'pre_val:{:.4f}'.format(prf_val[0]),
                      'recall_val:{:.4f}'.format(prf_val[1]),
                      'F1_val:{:4f}'.format(prf_val[2]),
                      'AUC_val:{:.4f}'.format(auc_val))
                
                epoch_store.append(epoch+1)
                acc_train_store.append(acc_train)       
                pre_train_store.append(prf_train[0])
                recall_train_store.append(prf_train[1])
                F1_train_store.append(prf_train[2])
                AUC_train_store.append(auc_train)
                acc_val_store.append(acc_val)
                pre_val_store.append(prf_val[0])
                recall_val_store.append(prf_val[1])
                F1_val_store.append(prf_val[2])
                AUC_val_store.append(auc_val)
                
                # save pth
                if acc_val > acc and epoch > 50:
                    acc = acc_val
                    if args.ckpt_path != '':
                        if not os.path.exists(args.ckpt_path):
                            os.makedirs(args.ckpt_path)
                        torch.save(model.state_dict(), fold_model_path)
                
                # Early Stopping
                if epoch > 50 and args.early_stopping == True:
                    if loss_val < best_val_loss:
                        best_val_loss = loss_val
                        current_patience = 0
                    else:
                        current_patience += 1
                    if current_patience >= args.early_stopping_patience:
                        print('Early Stopping!!! epoch：{}'.format(epoch))
                        break
        print("===================================================================",i,"_",j)
        data  = { 
              "epoch" : epoch_store ,
              "acc_train" : acc_train_store ,        
              "pre_train" : pre_train_store ,
              "recall_train" : recall_train_store ,
              "F1_train" : F1_train_store ,
              "AUC_train" : AUC_train_store ,
              "acc_val" : acc_val_store,
               "pre_val" : pre_val_store ,
              "recall_val" : recall_val_store ,
              "F1_val" : F1_val_store ,
              "AUC_val" : AUC_val_store  
        }
        
        
#         epoch_file_path =  f'../files/rois_ez/file_{i}_{j}_{count}.csv'
#         data_file = pd.DataFrame(data);
#         data_file.to_csv(epoch_file_path , index=False);
#         count=count+1;
        # test
#         print("Loading the Model for the {}-th Fold:... ...".format(i+1),
#               "Size of samples in the test set:{}".format(len(test_ind)))
#         model.load_state_dict(torch.load(fold_model_path))
        model.eval()
        
        with torch.set_grad_enabled(False):
            output, edge_weights = model(features, edge_index, edgenet_input)
        acc_test = torchmetrics_accuracy(output[test_ind], labels[test_ind])
        auc_test = torchmetrics_auc(output[test_ind], labels[test_ind])
        logits_test = output[test_ind].detach().cpu().numpy()
        correct_test = correct_num(logits_test, y[test_ind])
        prf_test =  prf(logits_test, y[test_ind])
        
        t_end = time.time()
        t = t_end - t_start
        print('Fold {} Results:'.format(i+1),
              'test acc:{:.4f}'.format(acc_test),
              'test_pre:{:.4f}'.format(prf_test[0]),
              'test_recall:{:.4f}'.format(prf_test[1]),
              'test_F1:{:.4f}'.format(prf_test[2]),
              'test_AUC:{:.4f}'.format(auc_test),
              'time:{:.3f}s'.format(t))
        
        correct = correct_test
        aucs[i] = auc_test
        prfs[i] = prf_test
        corrects[i] = correct
        test_num[i] = len(test_ind)
        

    if args.train == 0:
#         node_ftr = dataloader.get_node_features(train_ind)
#         edge_index, edgenet_input = dataloader.get_WL_inputs(nonimg)
#         edgenet_input = (edgenet_input - edgenet_input.mean(axis=0)) / edgenet_input.std(axis=0)
        node_ftr = dataloader.get_node_features(train_ind)
        n = node_ftr.shape[0]
        num_edge = n*(1+n)//2 - n
        pd_ftr_dim = nonimg.shape[1]
        edge_index = np.zeros([2, num_edge], dtype=np.int64) 
        edgenet_input = np.zeros([num_edge, 2*pd_ftr_dim], dtype=np.float32)
        model = GCN(input_dim = args.num_features,
                    nhid = args.hidden, 
                    num_classes = 2, 
                    ngl = args.ngl, 
                    dropout = args.dropout, 
                    edge_dropout = args.edropout, 
                    edgenet_input_dim = 2*nonimg.shape[1])
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
        
#         if args.cuda
        model
        features = torch.tensor(node_ftr, dtype=torch.float)
        edge_index = torch.tensor(edge_index, dtype=torch.long)
        edgenet_input = torch.tensor(edgenet_input, dtype=torch.float32)
        labels = torch.tensor(y, dtype=torch.long)
        fold_model_path = args.ckpt_path + "/fold{}.pth".format(i+1)
        
        model.load_state_dict(torch.load(fold_model_path))
        model.eval()
       
        with torch.set_grad_enabled(False):
     
            output, edge_weights = model(features, edge_index, edgenet_input)
        acc_test = torchmetrics_accuracy(output[test_ind], labels[test_ind])
        auc_test = torchmetrics_auc(output[test_ind], labels[test_ind])
        logits_test = output[test_ind].detach().cpu().numpy()
        correct_test = correct_num(logits_test, y[test_ind])
        prf_test =  prf(logits_test, y[test_ind])
        
        t_end = time.time()
        t = t_end - t_start
        print('Fold {} Results:'.format(i+1),
              'test acc:{:.4f}'.format(acc_test),
              'test_pre:{:.4f}'.format(prf_test[0]),
              'test_recall:{:.4f}'.format(prf_test[1]),
              'test_F1:{:.4f}'.format(prf_test[2]),
              'test_AUC:{:.4f}'.format(auc_test),
              'time:{:.3f}s'.format(t))
        
        correct = correct_test
        aucs[i] = auc_test
        prfs[i] = prf_test
        corrects[i] = correct
        test_num[i] = len(test_ind)

t2 = time.time()

print('\r\n======Finish Results for Nested 10-fold cross-validation======')
Nested10kCV_acc = np.sum(corrects) / np.sum(test_num)
Nested10kCV_auc = np.mean(aucs)
Nested10kCV_precision, Nested10kCV_recall, Nested10kCV_F1 = np.mean(prfs, axis=0)
print('Test:',
      'acc:{}'.format(Nested10kCV_acc),
      'precision:{}'.format(Nested10kCV_precision),
      'recall:{}'.format(Nested10kCV_recall),
      'F1:{}'.format(Nested10kCV_F1),
      'AUC:{}'.format(Nested10kCV_auc))
print('Total duration:{}'.format(t2 - t1))

    
    
   

Hyperparameters:
no_cuda: False
seed: 46
epochs: 2
lr: 0.001
weight_decay: 5e-05
hidden: 16
dropout: 0.2
atlas: ez
num_features: 2000
folds: 2
connectivity: correlation
max_degree: 3
ngl: 8
edropout: 0.3
train: 0
ckpt_path: ../folds/rois_ez_pth_b
early_stopping: True
early_stopping_patience: 20
cuda: False
  Loading dataset ...
['50002' '50003' '50004' ... '51585' '51606' '51607']
['50002' '50003' '50004' ... '51585' '51606' '51607']


C:\Users\hpw\AppData\Local\Temp\ipykernel_32700\837818515.py:157: RuntimeWarning: divide by zero encountered in arctanh
  norm_networks = [np.arctanh(mat) if not np.all(np.abs(mat) == 1) else mat for mat in all_networks]


y :  [0. 0. 0. ... 0. 0. 0.]
['50002' '50003' '50004' ... '51585' '51606' '51607']
train_ind , test_ind :  556 556
[ 633  991  626  661  848  705  875  990 1099  719  863 1054  914 1025
  794  535  753  703  733  581  836 1079  589  797  667  983  644  762
  630  530  925  907  834  759  767  660  772  929 1100 1046 1093  993
 1088  887  967  538  855  872 1075 1090  713  537 1086  843  831  587]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147

FileNotFoundError: [Errno 2] No such file or directory: '../folds/rois_ez_pth_b/fold1.pth'

In [ ]:
# pip install gradio
import torch
import torch.nn.functional as F



output1 = F.softmax(output, dim=1)
_, output = torch.max(output1, dim=1)

print(output)




In [ ]:
output = [tensor.item() for tensor in output]
for i in valid_ind: 
    print(subID[i-1]," ",output[i-1])

In [ ]:
import gradio as gr
def predict_model1(image):
    data = np.loadtxt(image)
    print(data.shape)
    file_name = os.path.basename(image)
    file_id = file_name.split('_')
    file_id = int(file_id[len(file_id)-3])
    j=0
    print(file_id)
    print(subID)
    for i in subID:
        print(i," ")
        if int(i) == int(file_id):
            return output[j]
        j = j + 1
    return 0
#     print(output)  
#     for i in subID:
#         if(i==file_id):
            
    
    
   
#     pred = loaded_model.predict(new_file)[0][0]
#     print(pred)
#     res = "Hurray, Person is Healthy!"
#     if(pred>0.5):
#         res = "Sorry, Person have ASD!"
#     return res

In [ ]:
gr.Interface(fn=predict_model1, inputs="file", outputs="label",title="<h1 style='margin-left:35%;width:70%; text-align: center;color: blue;'>Welcome to the ASD Model Prediction!</h1>",description="Upload an image file to make a prediction").launch(debug=True,share =True)

In [ ]:
print(len(raw_features))

In [ ]:
import torch
import torch.nn.functional as F

preds1 = torch.tensor([[-1.3694e-01, -3.0592e-01],
                       [-5.1673e-01, -9.9088e-02],
                       [-2.7485e-03,  7.0638e-02]])

preds2 = torch.tensor([[0.2, 0.8],
                       [0.6, 0.4],
                       [0.3, 0.7]])

# Define a hyperparameter to control the contribution of each dataset's predictions
alpha = 0.6  # Weight parameter (you can adjust this value)

# Combine the predictions using the weighted sum
combined_preds = alpha * preds1 + (1 - alpha) * preds2

# Apply softmax along axis 1 to convert logits to probabilities
probs = F.softmax(combined_preds, dim=1)
print("Probabilities:")
print(probs)


In [ ]:
_, predicted_labels = torch.max(combined_preds, dim=1)

In [ ]:
predicted_labels

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
csv_file_path0 = '../files/rois_ez_b/file_0_9_1.csv'
csv_file_path1 = '../files/rois_ez_b/file_1_9_2.csv'
csv_file_path2 = '../files/rois_ez_b/file_2_9_3.csv'
csv_file_path3 = '../files/rois_ez_b/file_3_9_4.csv'
csv_file_path4 = '../files/rois_ez_b/file_4_9_5.csv'
csv_file_path5 = '../files/rois_ez_b/file_5_9_6.csv'
csv_file_path6 = '../files/rois_ez_b/file_6_9_7.csv'
csv_file_path7 = '../files/rois_ez_b/file_7_9_8.csv'
csv_file_path8 = '../files/rois_ez_b/file_8_9_9.csv'
csv_file_path9 = '../files/rois_ez_b/file_9_9_10.csv'




all_fc_matrices_path_df0 = pd.read_csv(csv_file_path0)
all_fc_matrices_path_df1 = pd.read_csv(csv_file_path1)
all_fc_matrices_path_df2 = pd.read_csv(csv_file_path2)
all_fc_matrices_path_df3 = pd.read_csv(csv_file_path3)
all_fc_matrices_path_df4 = pd.read_csv(csv_file_path4)
all_fc_matrices_path_df5 = pd.read_csv(csv_file_path5)
all_fc_matrices_path_df6 = pd.read_csv(csv_file_path6)
all_fc_matrices_path_df7 = pd.read_csv(csv_file_path7)
all_fc_matrices_path_df8 = pd.read_csv(csv_file_path8)
all_fc_matrices_path_df9 = pd.read_csv(csv_file_path9)

In [ ]:


# Extract data from DataFrame
acc_train0 = all_fc_matrices_path_df0['acc_val'].tolist()
acc_train1 = all_fc_matrices_path_df1['acc_val'].tolist()
acc_train2 = all_fc_matrices_path_df2['acc_val'].tolist()
acc_train3 = all_fc_matrices_path_df3['acc_val'].tolist()
acc_train4 = all_fc_matrices_path_df4['acc_val'].tolist()
acc_train5 = all_fc_matrices_path_df5['acc_val'].tolist()
acc_train6 = all_fc_matrices_path_df6['acc_val'].tolist()
acc_train7 = all_fc_matrices_path_df7['acc_val'].tolist()
acc_train8 = all_fc_matrices_path_df8['acc_val'].tolist()
acc_train9 = all_fc_matrices_path_df9['acc_val'].tolist()


acc_train0 = [float(value.split('(')[1].split(')')[0]) for value in acc_train0]
acc_train1 = [float(value.split('(')[1].split(')')[0]) for value in acc_train1]
acc_train2 = [float(value.split('(')[1].split(')')[0]) for value in acc_train2]
acc_train3 = [float(value.split('(')[1].split(')')[0]) for value in acc_train3]
acc_train4 = [float(value.split('(')[1].split(')')[0]) for value in acc_train4]
acc_train5 = [float(value.split('(')[1].split(')')[0]) for value in acc_train5]
acc_train6 = [float(value.split('(')[1].split(')')[0]) for value in acc_train6]
acc_train7 = [float(value.split('(')[1].split(')')[0]) for value in acc_train7]
acc_train8 = [float(value.split('(')[1].split(')')[0]) for value in acc_train8]
acc_train9 = [float(value.split('(')[1].split(')')[0]) for value in acc_train9]



# acc_test = [float(value.split('(')[1].split(')')[0]) for value in acc_test]

epochs = range(1, len(acc_train) + 1)  # Assuming each entry corresponds to one epoch

# Plotting
fig = plt.figure(figsize=(30, 15))  # Adjust dimensions as needed

# Plotting
plt.plot(acc_train0, label='Train fold 0')
plt.plot(acc_train1, label='Train fold 1')
plt.plot(acc_train2, label='Train fold 2')
plt.plot(acc_train3, label='Train fold 3')
plt.plot(acc_train4, label='Train fold 4')
plt.plot(acc_train5, label='Train fold 5')
plt.plot(acc_train6, label='Train fold 6')
plt.plot(acc_train7, label='Train fold 7')
plt.plot(acc_train8, label='Train fold 8')
plt.plot(acc_train9, label='Train fold 9')


plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
csv_file_path = '../files/rois_cc200/file_0_9_1.csv'
all_fc_matrices_path_df = pd.read_csv(csv_file_path)

# Extract data from DataFrame
acc_train = all_fc_matrices_path_df['acc_train'].tolist()
acc_test = all_fc_matrices_path_df['acc_val'].tolist()
epochs = range(1, len(acc_train) + 1)  # Assuming each entry corresponds to one epoch

acc_train = [float(value.split('(')[1].split(')')[0]) for value in acc_train]
acc_test = [float(value.split('(')[1].split(')')[0]) for value in acc_test]

# Set figure size and background color
fig = plt.figure(figsize=(10, 6))  # Adjust dimensions as needed

# Plotting
plt.plot(epochs, acc_train, label='Train')
plt.plot(epochs, acc_test, label='Test')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
csv_file_path = '../files/rois_ez/file_0_9_1.csv'
all_fc_matrices_path_df = pd.read_csv(csv_file_path)

# Extract data from DataFrame
acc_train = all_fc_matrices_path_df['acc_train'].tolist()
acc_test = all_fc_matrices_path_df['acc_val'].tolist()
epochs = range(1, len(acc_train) + 1)  # Assuming each entry corresponds to one epoch

acc_train = [float(value.split('(')[1].split(')')[0]) for value in acc_train]
acc_test = [float(value.split('(')[1].split(')')[0]) for value in acc_test]

# Set figure size and background color
fig = plt.figure(figsize=(10, 6))  # Adjust dimensions as needed

# Plotting
plt.plot(epochs, acc_train, label='Train')
plt.plot(epochs, acc_test, label='Test')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
csv_file_path = '../files/rois_tt/file_0_9_1.csv'
all_fc_matrices_path_df = pd.read_csv(csv_file_path)

# Extract data from DataFrame
acc_train = all_fc_matrices_path_df['acc_train'].tolist()
acc_test = all_fc_matrices_path_df['acc_val'].tolist()
epochs = range(1, len(acc_train) + 1)  # Assuming each entry corresponds to one epoch

acc_train = [float(value.split('(')[1].split(')')[0]) for value in acc_train]
acc_test = [float(value.split('(')[1].split(')')[0]) for value in acc_test]

# Set figure size and background color
fig = plt.figure(figsize=(10, 6))  # Adjust dimensions as needed

# Plotting
plt.plot(epochs, acc_train, label='Train')
plt.plot(epochs, acc_test, label='Test')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
csv_file_path = '../files/rois_dosenbach160/file_0_9_1.csv'
all_fc_matrices_path_df = pd.read_csv(csv_file_path)

# Extract data from DataFrame
acc_train = all_fc_matrices_path_df['acc_train'].tolist()
acc_test = all_fc_matrices_path_df['acc_val'].tolist()
epochs = range(1, len(acc_train) + 1)  # Assuming each entry corresponds to one epoch

acc_train = [float(value.split('(')[1].split(')')[0]) for value in acc_train]
acc_test = [float(value.split('(')[1].split(')')[0]) for value in acc_test]

# Set figure size and background color
fig = plt.figure(figsize=(10, 6))  # Adjust dimensions as needed

# Plotting
plt.plot(epochs, acc_train, label='Train')
plt.plot(epochs, acc_test, label='Test')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
csv_file_path = '../files/rois_ho/file_0_9_1.csv'
all_fc_matrices_path_df = pd.read_csv(csv_file_path)

# Extract data from DataFrame
acc_train = all_fc_matrices_path_df['acc_train'].tolist()
acc_test = all_fc_matrices_path_df['acc_val'].tolist()
epochs = range(1, len(acc_train) + 1)  # Assuming each entry corresponds to one epoch

acc_train = [float(value.split('(')[1].split(')')[0]) for value in acc_train]
acc_test = [float(value.split('(')[1].split(')')[0]) for value in acc_test]

# Set figure size and background color
fig = plt.figure(figsize=(10, 6))  # Adjust dimensions as needed

# Plotting
plt.plot(epochs, acc_train, label='Train')
plt.plot(epochs, acc_test, label='Test')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
csv_file_path = '../files/rois_ez/file_0_9_1.csv'
all_fc_matrices_path_df = pd.read_csv(csv_file_path)

# Extract data from DataFrame
pre = all_fc_matrices_path_df['pre_train'].tolist()
recall = all_fc_matrices_path_df['recall_train'].tolist()
epochs = range(1, len(acc_train) + 1)  # Assuming each entry corresponds to one epoch



# Set figure size and background color
fig = plt.figure(figsize=(10, 6))  # Adjust dimensions as needed

# Plotting
plt.plot( pre, label='pre_train')
plt.plot( recall, label='recall_train')
plt.title('Model accuracy')
plt.ylabel('pre_recall_train')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
csv_file_path = '../files/rois_ez/file_0_9_1.csv'
all_fc_matrices_path_df = pd.read_csv(csv_file_path)

# Extract data from DataFrame
pre = all_fc_matrices_path_df['pre_val'].tolist()
recall = all_fc_matrices_path_df['recall_val'].tolist()
epochs = range(1, len(acc_train) + 1)  # Assuming each entry corresponds to one epoch



# Set figure size and background color
fig = plt.figure(figsize=(10, 6))  # Adjust dimensions as needed

# Plotting
plt.plot( pre, label='pre_train')
plt.plot( recall, label='recall_train')
plt.title('Model accuracy')
plt.ylabel('pre_recall_val')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
csv_file_path = '../files/rois_ez/file_0_9_1.csv'
all_fc_matrices_path_df = pd.read_csv(csv_file_path)

# Extract data from DataFrame
f1 = all_fc_matrices_path_df['F1_val'].tolist()

epochs = range(1, len(acc_train) + 1)  # Assuming each entry corresponds to one epoch



# Set figure size and background color
fig = plt.figure(figsize=(10, 6))  # Adjust dimensions as needed

# Plotting
plt.plot( f1, label='pre_train')

plt.title('f1 score')
plt.ylabel('f1')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
csv_file_path = '../files/rois_cc200/file_0_9_1.csv'
all_fc_matrices_path_df = pd.read_csv(csv_file_path)

# Extract data from DataFrame
AUC_val = all_fc_matrices_path_df['AUC_val'].tolist()

epochs = range(1, len(AUC_val ) + 1)  # Assuming each entry corresponds to one epoch

AUC_val = [float(value.split('(')[1].split(')')[0]) for value in AUC_val]


# Set figure size and background color
fig = plt.figure(figsize=(10, 6))  # Adjust dimensions as needed

# Plotting
plt.plot(epochs, AUC_val, label='AUC_val')

plt.title('AUC_val')
plt.ylabel('AUC_val')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()
